# 데이터 로딩 및 확인

In [4]:
import glob, os
import pandas as pd

all_files = glob.glob(os.path.join(r'.\topics', '*.data'))
print(type(all_files))
print(all_files[:2])

df = pd.read_table(all_files[0], encoding='latin1')
print(df.shape)
print(df.head())
# print(df.to_string())

<class 'list'>
['.\\topics\\accuracy_garmin_nuvi_255W_gps.txt.data', '.\\topics\\bathroom_bestwestern_hotel_sfo.txt.data']
(66, 1)
                      , and is very, very accurate .
0   but for the most part, we find that the Garmi...
1   This function is not accurate if you don't le...
2   It provides immediate alternatives if the rou...
3   I've used other GPS units, as well as GPS bui...
4   It got me from point A to point B with 100% a...


In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [9]:
path = r'.\topics'
all_files = glob.glob(os.path.join(path, '*.data'))
filenames = [] # 파일명 
opinion_text = [] # 리뷰
for file in all_files:
    df = pd.read_table(file, encoding='latin1')
    filename = file.split('\\')[-1]
    filename = filename.split('.')[0]
    filenames.append(filename)
    opinion_text.append(df.to_string()) # to_string() : 데이터 프레임을 문자열로 바꿔줌

document_df = pd.DataFrame({'filename':filenames, 'opinion_text':opinion_text})
print(document_df.shape)
display(document_df.head())

(51, 2)


filename  \
0   accuracy_garmin_nuvi_255W_gps   
1  bathroom_bestwestern_hotel_sfo   
2      battery-life_amazon_kindle   
3      battery-life_ipod_nano_8gb   
4     battery-life_netbook_1005ha   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

# TfidfVectorizer의 tokenizer 인자로 사용될 어근 변환 함수 선언

In [12]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [16]:
print(ord('A'))
print(chr(65))

65
A


- translate() 함수
>- 문자열 내 특정 문자를 다른 문자로 일괄 변경하는 함수
>- 매개변수로 변경하고자 하는 정보를 딕셔너리로 전달 {원본 문자 유니코드:변경 문자 유니코드}

In [18]:
# maketrans() : 원래 문자와 바꿀 문자에 대한 맵핑정보(딕셔너리)를 만든다.
trans = str.maketrans('[],','   ')
# 대괄호와 콤마를 공백문자로 바꿈
print(trans)
text = '[고양이,강아지,다람쥐,토끼]'
text = text.translate(trans)
print(text)

{91: 32, 93: 32, 44: 32}
 고양이 강아지 다람쥐 토끼 


In [19]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

# ord() : 매개변수로 전달된 문자에 해당하는 유니코드를 반환 <--> chr() 
remove_punct_dict = {ord(punct):None for punct in string.punctuation}
lemmar = WordNetLemmatizer()

# 입력된 문장을 받아서 -> 소문자 변환 -> 구두점 제거 -> 단어 단위로 토큰화 -> 어근 변환
def LemNormalize(text):
    tokens = nltk.word_tokenize(text.lower().translate(remove_punct_dict)) # translate 함수로 구두점을 제거
    return [lemmar.lemmatize(token) for token in tokens]
    

# TF-IDF 기반의 피처 벡터화
- min_df : 전체 문서에 걸쳐 너무 낮은 빈도를 갖는 단어 피처를 제외
>- 정수 : 지정한 개수 이하로 나타나는 단어는 피처 추출에서 제외
>- 실수 (0.0~1.0) : 지정한 백분율 이하 빈도수까지의 단어는 피처 추출에서 제외 (ex. 0.05 -> 하위 5% 이하의 빈도수를 가지는 단어는 피처 추출에서 제외)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

stopwords = nltk.corpus.stopwords.words('english')
tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords, ngram_range=(1,2), min_df=0.05, max_df=0.85)
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

C:\Users\user\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\user\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arent', 'couldnt', 'didnt', 'doe', 'doesnt', 'dont', 'ha', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shant', 'shes', 'shouldnt', 'shouldve', 'thatll', 'wa', 'wasnt', 'werent', 'wont', 'wouldnt', 'youd', 'youll', 'youre', 'youve'] not in stop_words.
  warnings.warn(


In [23]:
print(feature_vect.shape)
print(feature_vect[0].toarray())

(51, 5014)
[[0.04787771 0.         0.         ... 0.         0.         0.        ]]


# 군집 분석

## 군집 수 5개로 군집

In [24]:
from sklearn.cluster import KMeans

km_cluster = KMeans(n_clusters=5, random_state=0, max_iter=10000)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_center = km_cluster.cluster_centers_

C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [27]:
document_df['cluster_label'] = cluster_label
display(document_df.head())

filename  \
0   accuracy_garmin_nuvi_255W_gps   
1  bathroom_bestwestern_hotel_sfo   
2      battery-life_amazon_kindle   
3      battery-life_ipod_nano_8gb   
4     battery-life_netbook_1005ha   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [28]:
display(document_df[document_df['cluster_label']==0]['filename'])

1     bathroom_bestwestern_hotel_sfo
30       rooms_bestwestern_hotel_sfo
31           rooms_swissotel_chicago
32           room_holiday_inn_london
Name: filename, dtype: object

In [29]:
display(document_df[document_df['cluster_label']==1]['filename'])

2      battery-life_amazon_kindle
3      battery-life_ipod_nano_8gb
4     battery-life_netbook_1005ha
19        keyboard_netbook_1005ha
26     performance_netbook_1005ha
41       size_asus_netbook_1005ha
42            sound_ipod_nano_8gb
44                 speed_windows7
Name: filename, dtype: object

In [30]:
display(document_df[document_df['cluster_label']==2]['filename'])

0       accuracy_garmin_nuvi_255W_gps
5               buttons_amazon_kindle
8     directions_garmin_nuvi_255W_gps
9        display_garmin_nuvi_255W_gps
10      eyesight-issues_amazon_kindle
11                  features_windows7
12                fonts_amazon_kindle
23           navigation_amazon_kindle
33     satellite_garmin_nuvi_255W_gps
34        screen_garmin_nuvi_255W_gps
35               screen_ipod_nano_8gb
36              screen_netbook_1005ha
43         speed_garmin_nuvi_255W_gps
48       updates_garmin_nuvi_255W_gps
49                video_ipod_nano_8gb
50         voice_garmin_nuvi_255W_gps
Name: filename, dtype: object

In [31]:
display(document_df[document_df['cluster_label']==3]['filename'])

13            food_holiday_inn_london
14             food_swissotel_chicago
15         free_bestwestern_hotel_sfo
20     location_bestwestern_hotel_sfo
21        location_holiday_inn_london
24      parking_bestwestern_hotel_sfo
27                price_amazon_kindle
28           price_holiday_inn_london
38      service_bestwestern_hotel_sfo
39         service_holiday_inn_london
40    service_swissotel_hotel_chicago
45        staff_bestwestern_hotel_sfo
46            staff_swissotel_chicago
Name: filename, dtype: object

In [32]:
display(document_df[document_df['cluster_label']==4]['filename'])

6          comfort_honda_accord_2008
7          comfort_toyota_camry_2007
16     gas_mileage_toyota_camry_2007
17        interior_honda_accord_2008
18        interior_toyota_camry_2007
22         mileage_honda_accord_2008
25     performance_honda_accord_2008
29         quality_toyota_camry_2007
37           seats_honda_accord_2008
47    transmission_toyota_camry_2007
Name: filename, dtype: object

## 군집 수 3개로 군집

In [33]:
from sklearn.cluster import KMeans

km_cluster = KMeans(n_clusters=3, random_state=0, max_iter=10000)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_center = km_cluster.cluster_centers_

C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [34]:
document_df['cluster_label'] = cluster_label
display(document_df.head())

filename  \
0   accuracy_garmin_nuvi_255W_gps   
1  bathroom_bestwestern_hotel_sfo   
2      battery-life_amazon_kindle   
3      battery-life_ipod_nano_8gb   
4     battery-life_netbook_1005ha   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [35]:
display(document_df[document_df['cluster_label']==0]['filename'])

0       accuracy_garmin_nuvi_255W_gps
2          battery-life_amazon_kindle
3          battery-life_ipod_nano_8gb
4         battery-life_netbook_1005ha
5               buttons_amazon_kindle
8     directions_garmin_nuvi_255W_gps
9        display_garmin_nuvi_255W_gps
10      eyesight-issues_amazon_kindle
11                  features_windows7
12                fonts_amazon_kindle
19            keyboard_netbook_1005ha
23           navigation_amazon_kindle
26         performance_netbook_1005ha
27                price_amazon_kindle
33     satellite_garmin_nuvi_255W_gps
34        screen_garmin_nuvi_255W_gps
35               screen_ipod_nano_8gb
36              screen_netbook_1005ha
41           size_asus_netbook_1005ha
42                sound_ipod_nano_8gb
43         speed_garmin_nuvi_255W_gps
44                     speed_windows7
48       updates_garmin_nuvi_255W_gps
49                video_ipod_nano_8gb
50         voice_garmin_nuvi_255W_gps
Name: filename, dtype: object

In [36]:
display(document_df[document_df['cluster_label']==1]['filename'])

6          comfort_honda_accord_2008
7          comfort_toyota_camry_2007
16     gas_mileage_toyota_camry_2007
17        interior_honda_accord_2008
18        interior_toyota_camry_2007
22         mileage_honda_accord_2008
25     performance_honda_accord_2008
29         quality_toyota_camry_2007
37           seats_honda_accord_2008
47    transmission_toyota_camry_2007
Name: filename, dtype: object

In [37]:
display(document_df[document_df['cluster_label']==2]['filename'])

1      bathroom_bestwestern_hotel_sfo
13            food_holiday_inn_london
14             food_swissotel_chicago
15         free_bestwestern_hotel_sfo
20     location_bestwestern_hotel_sfo
21        location_holiday_inn_london
24      parking_bestwestern_hotel_sfo
28           price_holiday_inn_london
30        rooms_bestwestern_hotel_sfo
31            rooms_swissotel_chicago
32            room_holiday_inn_london
38      service_bestwestern_hotel_sfo
39         service_holiday_inn_london
40    service_swissotel_hotel_chicago
45        staff_bestwestern_hotel_sfo
46            staff_swissotel_chicago
Name: filename, dtype: object

# 군집별 핵심단어 추출하기
- cluster_centers_
>- 단어 피처가 군집의 중심으로부터 얼마나 가깝게 있는지를 군집 내의 피처들과의 상대 위치로 나타낸 값
>- 0~1 사이의 값으로 1에 가까울수록 중심과 가까운 것을 의미

In [40]:
cluster_centers = km_cluster.cluster_centers_
# (3, 5015)
# 3 : 군집 수
# 5014 : 피처(단어)의 수
print(cluster_centers.shape)
print(cluster_center[0][:10])

(3, 5014)
[0.00993046 0.         0.00127226 0.         0.         0.00051839
 0.         0.00102318 0.00125139 0.        ]


## 군집별 상위 n개의 핵심단어 추출 함수 정의
- 매개변수 설명
>- cluster_model : KMeans 모델 객체
>- cluster_data : 리뷰 문서 및 군집 결과 값을 가지고 있는 DataFrame 객체
>- feature_names : 피처 카운터 벡터화 된 각 피처(단어)의 이름
>- cluster_num : 군집 수
>- top_n_features : 군집별 확인하고자 하는 상위 n개의 피처(단어) 수

In [54]:
def get_cluster_details(cluster_model, cluster_data, feature_names, cluster_num, top_n_features=10):
    cluster_details = {}
    centroid_feature_ordered_idx = cluster_model.cluster_centers_.argsort()[:,::-1]

    for num in range(cluster_num): # 군집 수 만큼 반복 수행
        # {0:{}}
        cluster_details[num] = {} # 개별 군집 정보를 담을 사전객체 초기화
        # {0:{'cluster':0}
        cluster_details[num]['cluster'] = num
        top_feature_indexes = centroid_feature_ordered_idx[num, :top_n_features]
        top_features = [feature_names[ind] for ind in top_feature_indexes]
        # 군집의 중심으로부터 가까운 상위 n개 단어의 중심점으로부터의 거리 값 목록
        top_features_values = cluster_model.cluster_centers_[num, top_feature_indexes].tolist()

        cluster_details[num]['top_features'] = top_features
        cluster_details[num]['top_features_value'] = top_features_values
        filenames = cluster_data[cluster_data['cluster_label']==num]['filename']
        filenames = filenames.values.tolist() # 시리즈 객체.values() -> np 다차원 배열 -> 리스트
        cluster_details[num]['filenames'] = filenames

    return cluster_details

In [55]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print(f'#### Cluster: {cluster_num} ####')
        print(f'Top Features: {cluster_detail["top_features"]}')
        print(f'Reveiw 파일명: {cluster_detail["filenames"][:5]}')
        print('='*30)

In [56]:
feature_names = tfidf_vect.get_feature_names_out()
cluster_details = get_cluster_details(km_cluster, document_df, feature_names, 3)
print_cluster_details(cluster_details)

#### Cluster: 0 ####
Top Features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'video', 'direction', 'feature', 'voice']
Reveiw 파일명: ['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_kindle']
#### Cluster: 1 ####
Top Features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reveiw 파일명: ['comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_honda_accord_2008', 'interior_toyota_camry_2007']
#### Cluster: 2 ####
Top Features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reveiw 파일명: ['bathroom_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'food_swissotel_chicago', 'free_bestwestern_hotel_sfo', 'location_bestwestern_hotel_sfo']
